In [1]:
#############################################################
# IMPORTAMOS LAS LIBRERIAS QUE USAREMOS EN LAS EXTRACCIONES #
#############################################################

from bs4 import BeautifulSoup as soup
import requests
import urllib.request   
import pandas as pd

from urllib.request import urlopen
from bs4 import BeautifulSoup as soup



## LECTURA DEL EXCEL 

In [265]:
gen_abreviado1=[] # creamos una lista para añadir la columna Gen Abrev, de la hoja C1.
DF = pd.read_excel('Genes identificados con vecinos.xlsx', sheet_name="C1",skiprows=4)
genasociado= DF["Gen Abrev"]

for i in genasociado:
    gen_abreviado1.append(i)
#print(gen_abreviado1)
len(gen_abreviado1) 


89

In [266]:
# REALIZAMOS LA LECTURA PARA LAS HOJAS C2 HASTA C11 DEL EXCEL
hojas_excel = ["C2","C3","C4-M","C5","C6","C7-J","C8-N","C9","C10","C11"]
gen_abreviado2=[] # CREAMOS UNA LISTA PARA ALMACENAR TODOS LOS DATOS DE LA COLUMNA Gen Abrev
for i in hojas_excel:
    datos = pd.read_excel('Genes identificados con vecinos.xlsx', sheet_name=i,skiprows=1)
    genabreviado=datos["Gen Abrev"]
    for nombre in genabreviado:
        gen_abreviado2.append(nombre)
#print(gen_abreviado2)
len(gen_abreviado2)

2442

In [270]:
# SE JUNTAN TODOS LOS DATOS DE LA COLUMNA Gen Abrev EN total_Gen_Abrev

total_Gen_Abrev = gen_abreviado1 + gen_abreviado2
total_Gen_Abrev
total_Gen_Abrev.remove('TRNAC-GCA ') # ELIMINAMOS  EL DATO "TRNAC-GCA "PORQUE SE REPITE Y TIENE UN ESPACIO EN BLANCO
#print(total_Gen_Abrev)
len(total_Gen_Abrev)

2530

In [271]:
##Guardando un archivo de formato data frame
#Creamos un diccionario data
data = {'Gen abrev': total_Gen_Abrev}
# Luego creamos un dataframe y especificamos las columnas.
df = pd.DataFrame(data, columns = ['Gen abrev'])

In [272]:
DF2=df["Gen abrev"].unique() # unique(): SE ELIMINAN LAS REPETICIONES DE LAS PALABRAS.
#h.head(22)
oficial_Gen = pd.DataFrame(DF2, columns = ['Gen abrev'])
#oficial_Gen.value_counts()
oficial_Gen  # Data de genes limpios

,Gen abrev
0,TRNAG-UCC
1,ROBO1
2,TRNAC-GCA
3,TRNAC-ACA
4,MIR2286
...,...
2450,LOC107132968
2451,REXO4
2452,ADAMTS13
2453,CACFD1


## CREAMOS UN NUEVO EXCEL CON TODA LA COLUMNA Gen Abrev LIMPIA

In [274]:
### creación del excel a partir del Dataframe
from pandas import ExcelWriter
writer = ExcelWriter('./Gen_abrev.xlsx')
oficial_Gen.to_excel(writer, 'C1', index=True)
writer.save()

# EL NOMBRE DEL ARCHIVO EXCEL SERÁ "Gen_abrev.xlsx"

In [279]:
#################################################################################
# LECTURA DEl NUEVO EXCEL  (las primeras 818 líneas para esta primera parte)    #
#################################################################################

DF = pd.read_excel('Gen_abrev.xlsx', sheet_name="C1",skiprows=0)
DF
genasociado= DF["Gen abrev"]
genasociado=genasociado[0:818] # solo extraemos las primeras 818 filas
genasociado

0      TRNAG-UCC
1          ROBO1
2      TRNAC-GCA
3      TRNAC-ACA
4        MIR2286
         ...    
813       TUBA1B
814       LMBR1L
815          DHH
816       RHEBL1
817        KMT2D
Name: Gen abrev, Length: 818, dtype: object

### CREACIÓN DE COLUMNAS

In [320]:
########################################################
# COLUMNA1 Gen Abrev                                   #
########################################################

gen_abreviado1=[]

for i in genasociado:
    gen_abreviado1.append(i)
#print(gen_abreviado1)
len(gen_abreviado1)

818

In [281]:
############################################################
# CAMBIANDO ALGUNOS CARACTERES PARA LA BUSQUEDA            #
############################################################

#para crear los links correspondientess de busqueda tenemos que hacer una modificaciones
lista=[]
for genes in gen_abreviado1:
    lista.append(genes.replace(" ", "+"))
#print(lista)

lista1=[]
for genes in lista:
    lista1.append(genes.replace("(","%28"))
#print(lista1)

query=[]
for genes in lista1:
    query.append(genes.replace(")","%29"))
#print(query)    
len(query)   

818

In [329]:
####################################################
# CREACION DE LINKS(PRIMERA BÚSQUEDA)              #
####################################################

links=[]

for i in query:
    links.append("https://www.uniprot.org/uniprot/?query="+ str(i) + "+Bos+taurus&sort=score")
#print(links)
len(links)

818

In [ ]:
####################################################
# COLUMNA2 ENTRY                                   #
####################################################

# IMPORTAMOS LAS LIBRERIAS QUE USAREMOS 

import urllib.request, urllib.parse, urllib.error #(solicitud,analizar,error)
import re
import ssl  
ctx = ssl.create_default_context()                    #crear un contexto predeterminado
ctx.check_hostname = False                            #comprobar el nombre de host
ctx.verify_mode = ssl.CERT_NONE                       #modo de verificación

##################################
# PRIMERAS 100 FILAS             #
##################################

# EN ESTA PRIMERA PARTE REALIZAREMOS LA EXTRACCIÓN DE DATOS DE 100 en 100.


LISTA_ENTRY_1 = []  # Creamos una lista vacia para almacenar los 100 primeros datos.

for codigo in links[0:100]:
    url = codigo # el codigo es cada link que esta en la lista links
    html = urllib.request.urlopen(url, context=ctx).read()  # Al html se le asigna toda la estructura que tiene el codigo(link) asignado
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) # realiza la busqueda de lo que queremos extraer y lo almacena en lista.
    if len(link)==0: # si no tiene lo que estamos solicitando a la lista creada agregará "Na"
        LISTA_ENTRY_1.append("Na")
    else:
        objeto=link[0].decode() # en caso de si tener nuestro pedido, realiza na decodifación y lo almacena en la variable objeto.
        LISTA_ENTRY_1 = LISTA_ENTRY_1 +[objeto] # Se va agregando a la lista.
#print(LISTA_ENTRY_1)   
len(LISTA_ENTRY_1)

# Este paso se realiza para las siguientes filas.
'''
##################################
# SIGUIENTES 100 FILAS           #
##################################

LISTA_ENTRY_2 = []

for codigo in links[100:200]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_2.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_2 = LISTA_ENTRY_2 +[objeto] 
#print(LISTA_ENTRY_2)   
print(len(LISTA_ENTRY_2))

##################################
# SIGUIENTES 100 FILAS           #
##################################


LISTA_ENTRY_3 = []

for codigo in links[200:300]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_3.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_3 = LISTA_ENTRY_3 +[objeto] 
print(LISTA_ENTRY_3)   
print(len(LISTA_ENTRY_3))


##################################
# SIGUIENTES 100 FILAS           #
##################################

LISTA_ENTRY_4 = []

for codigo in links[300:400]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_4.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_4 = LISTA_ENTRY_4 +[objeto] 
print(LISTA_ENTRY_4)   
print(len(LISTA_ENTRY_4))


##################################
# SIGUIENTES 100 FILAS           #
##################################

LISTA_ENTRY_5 = []

for codigo in links[400:500]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_5.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_5 = LISTA_ENTRY_5 +[objeto] 
print(LISTA_ENTRY_5)   
print(len(LISTA_ENTRY_5))

##################################
# SIGUIENTES 100 FILAS           #
##################################

LISTA_ENTRY_6 = []

for codigo in links[500:600]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_6.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_6 = LISTA_ENTRY_6 +[objeto] 
print(LISTA_ENTRY_6)   
print(len(LISTA_ENTRY_6))

##################################
# SIGUIENTES 100 FILAS           #
##################################

LISTA_ENTRY_7 = []

for codigo in links[600:700]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_7.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_7 = LISTA_ENTRY_7 +[objeto] 
print(LISTA_ENTRY_7)   
print(len(LISTA_ENTRY_7))

##################################
# SIGUIENTES 100 FILAS           #
##################################

LISTA_ENTRY_8 = []

for codigo in links[700:818]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<a href=".uniprot.([a-zA-Z].*?)"', html) 
    if len(link)==0:
        LISTA_ENTRY_8.append("Na")
    else:
        objeto=link[0].decode()
        LISTA_ENTRY_8 = LISTA_ENTRY_8 +[objeto] 
print(LISTA_ENTRY_8)   
print(len(LISTA_ENTRY_8))
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

LISTA_ENTRY = LISTA_ENTRY_1+LISTA_ENTRY_2+LISTA_ENTRY_3+LISTA_ENTRY_4+LISTA_ENTRY_5+LISTA_ENTRY_6+LISTA_ENTRY_7+LISTA_ENTRY_8
#print(LISTA_ENTRY)
len(LISTA_ENTRY) # Viene a ser la lista completa o total de nuestro requerimiento.


In [295]:
#######################################################
# COLUMNA3 ENTRY NAME                                 #
#######################################################

##################################
# PRIMERAS 100 FILAS             #
##################################


lista1 = [] # creamos una primera lista para almacenar las 100 primeras filas.

for codigo in links[0:100]: # se realiza la busqueda de los 100 primeros links
    url = codigo # cada codigo(link) se almacena en la variable url.
    html = urllib.request.urlopen(url, context=ctx).read()  # de cada codigo(link) se almacena la estructura html en "html"
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) # realiza la búsqueda a lo que solicitamos.
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista1.append('Na')
    else:
        objeto=link[0].decode() # en caso de tener información realiza una decodificación y se asigna a la variable objeto.
        lista1=lista1 +[objeto] # luego se va almacenando en la lista1 que fue creada.
print(lista1)    
print(len(lista1))

# Estos pasos se realizan para las demás filas

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

'''
lista2 = []

for codigo in links[100:200]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista2.append('Na')
    else:
        objeto=link[0].decode()
        lista2=lista2 +[objeto] 
print(lista2)    
print(len(lista2))


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################


lista3 = []

for codigo in links[200:300]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista3.append('Na')
    else:
        objeto=link[0].decode()
        lista3=lista3 +[objeto] 
print(lista3)    
print(len(lista3))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################


lista4 = []

for codigo in links[300:400]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista4.append('Na')
    else:
        objeto=link[0].decode()
        lista4=lista4 +[objeto] 
print(lista4)    
print(len(lista4))


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################


lista5 = []

for codigo in links[400:500]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista5.append('Na')
    else:
        objeto=link[0].decode()
        lista5=lista5 +[objeto] 
print(lista5)    
print(len(lista5))


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################


lista6 = []

for codigo in links[500:600]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista6.append('Na')
    else:
        objeto=link[0].decode()
        lista6=lista6 +[objeto] 
print(lista6)    
print(len(lista6))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################


lista7 = []

for codigo in links[600:700]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista7.append('Na')
    else:
        objeto=link[0].decode()
        lista7=lista7 +[objeto] 
print(lista7)    
print(len(lista7))


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################


lista8 = []

for codigo in links[700:818]:
    url = codigo
    html = urllib.request.urlopen(url, context=ctx).read()  
    link = re.findall(b'<td>([A-Z].*?)<.td>', html) 
    if len(link)==0: # si no hay información en el link agrega "Na"
        lista8.append('Na')
    else:
        objeto=link[0].decode()
        lista8=lista8 +[objeto] 
print(lista8)    
len(lista8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

LISTA_ENTRY_NAME = lista1 + lista2 + lista3 + lista4 + lista5 + lista6 + lista7 + lista8
print(LISTA_ENTRY_NAME)
len(LISTA_ENTRY_NAME)


['Na', 'UBP33_BOVIN', 'Na', 'Na', 'Na', 'Na', 'Na', 'A0A4W2F3U1_BOBOX', 'IFT57_BOVIN', 'E1BCC8_BOVIN', 'A0A3Q1LHR1_BOVIN', 'MYC_BOVIN', 'F6RVN5_BOVIN', 'P85A_BOVIN', 'A0A4W2HUI0_BOBOX', 'Na', 'Na', 'Na', 'F1MQI2_BOVIN', 'PCGF1_BOVIN', 'Na', 'Na', 'OPRM_BOVIN', 'COL11_BOVIN', 'E1BCB2_BOVIN', 'Na', 'A0A3Q1M347_BOVIN', 'Na', 'ADIPO_BOVIN', 'A0A3Q1M1J3_BOVIN', 'IF4A2_BOVIN', 'Na', 'KNG1_BOVIN', 'HRG_BOVIN', 'FETUB_BOVIN', 'FETUA_BOVIN', 'Na', 'DJB11_BOVIN', 'TBCC1_BOVIN', 'CRYGS_BOVIN', 'A0A3Q1LYT4_BOVIN', 'A0A4W2D3G5_BOBOX', 'Na', 'TRA2B_BOVIN', 'Na', 'DHX9_BOVIN', 'CXA1_BOVIN', 'LIPC_BOVIN', 'TM41A_BOVIN', 'M3K13_BOVIN', 'E1BMH4_BOVIN', 'F1MII5_BOVIN', 'PERL_BOVIN', 'CDK2_BOVIN', 'APC13_BOVIN', 'AMOL2_BOVIN', 'A0A3Q1N0Q2_BOVIN', 'F1MQ45_BOVIN', 'RAB6B_BOVIN', 'LALBA_BOVIN', 'A0A3Q1NEQ0_BOVIN', 'SMCA4_BOVIN', 'A6QLV0_BOVIN', 'E1BEG7_BOVIN', 'A0A4W2EA77_BOBOX', 'A2VDQ3_BOVIN', 'F1MN54_BOVIN', 'CAV1_BOVIN', 'TFF3_BOVIN', 'F6QX08_BOVIN', 'ESR1_BOVIN', 'F1MYZ1_BOVIN', 'F1MYY9_BOVIN', 'A0A3Q1M

818

In [297]:
#######################################################
# OBTENIENDO URLS (SEGUNDA BUSQUEDA)                  #
#######################################################

urls=[] # CREAMOS UNA NUEVA LISTA LLAMADA urls DONDE SE REALIZARA LA SEGUNDA BÚSQUEDA

for i in LISTA_ENTRY: # ESTA urls SE VA A CREAR USANDO LA LISTA "LISTA_ENTRY".
    if i=="Na": # SI UN ELEMENTO DE LISTA_ENTRY ES "Na" ENTONCES SE VA A AÑADIR "Na" YA QUE NO HAY PARA PASAR OTRO LINK.
        urls.append("Na")
    else:
        urls.append("https://www.uniprot.org/uniprot/"+str(i)) #EN CASO DE SI TENER UN ELEMENTO DISTINTO DE "Na" SE VA AGREGAR
                                                              # EL LINK DONDE SE VA A A AGREGAR EL ELEMENTO CONVERTIDO EN string.
print(urls) # SE MUESTRAN TODAS LAS urls CREADAS 
len(urls) 

['Na', 'https://www.uniprot.org/uniprot/A6QNM7', 'Na', 'Na', 'Na', 'Na', 'Na', 'https://www.uniprot.org/uniprot/A0A4W2F3U1', 'https://www.uniprot.org/uniprot/Q5EA95', 'https://www.uniprot.org/uniprot/E1BCC8', 'https://www.uniprot.org/uniprot/A0A3Q1LHR1', 'https://www.uniprot.org/uniprot/Q2HJ27', 'https://www.uniprot.org/uniprot/F6RVN5', 'https://www.uniprot.org/uniprot/P23727', 'https://www.uniprot.org/uniprot/A0A4W2HUI0', 'Na', 'Na', 'Na', 'https://www.uniprot.org/uniprot/F1MQI2', 'https://www.uniprot.org/uniprot/Q2YDF9', 'Na', 'Na', 'https://www.uniprot.org/uniprot/P79350', 'https://www.uniprot.org/uniprot/Q17QH6', 'https://www.uniprot.org/uniprot/E1BCB2', 'Na', 'https://www.uniprot.org/uniprot/A0A3Q1M347', 'Na', 'https://www.uniprot.org/uniprot/Q3Y5Z3', 'https://www.uniprot.org/uniprot/A0A3Q1M1J3', 'https://www.uniprot.org/uniprot/Q3SZ65', 'Na', 'https://www.uniprot.org/uniprot/P01044', 'https://www.uniprot.org/uniprot/P33433', 'https://www.uniprot.org/uniprot/Q58D62', 'https://www.

818

In [309]:
###########################################
# COLUMNA4 PROTEIN                        #
###########################################

##################################
# PRIMERAS 100 FILAS             #
##################################


PROTEINA1=[] # CREAMOS UNA LISTA PARA ALMACENAR LAS 100 PRIMERAS FILAS DE LA COLUMNA PROTEIN

for i in urls[0:100]: # SE VA A REALIZAR UNA BUSQUEDA POR CADA ELEMENTO DE urls
    if i=="Na": # EN CASO DE QUE la url SEA IGUAL A "Na" NO SE REALIZA LA BUSQUEDA POR LO TANTO LO AGREGA COMO "Na"
        PROTEINA1.append("Na")

    else:        # SI HAY url DIFERENTE DE Na REALIZA LO SIGUIENTE
        url = i 
        html = urllib.request.urlopen(url, context=ctx).read()  # ALMACENA LA ESTRUCTURA DE html EN LA VARIABLE html.
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) # REALIZA LA BÚSQUEDA 
        for link in links:
            nuevo=link.decode() # REALIZA LA DECODIFICACIÓN DE CADA link Y LO ALMACENA EN LA VARIABLE nuevo.
            PROTEINA1=PROTEINA1+[nuevo] # DESPÚES DE REALIZARSE LA DECODIFICACIÓN SE ALMACENA EN LA LISTA "PROTEINA1"
            
print(PROTEINA1) # SE MUESTRA LA INFORMACION DE LAS 100 PRIMERAS FILAS DE LA COLUMNA PROTEINA.
len(PROTEINA1)
'''
# ESTOS PASOS SE REALIZAN PARA LAS SIGUIENTES FILAS.

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA2=[]

for i in urls[100:200]:
    if i=="Na":
        PROTEINA2.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA2=PROTEINA2+[nuevo]
            
print(PROTEINA2)
print(len(PROTEINA2))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA3=[]

for i in urls[200:300]:
    if i=="Na":
        PROTEINA3.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA3=PROTEINA3+[nuevo]
            
print(PROTEINA3)
print(len(PROTEINA3))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA4=[]

for i in urls[300:400]:
    if i=="Na":
        PROTEINA4.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA4=PROTEINA4+[nuevo]
            
print(PROTEINA4)
len(PROTEINA4)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA5=[]

for i in urls[400:500]:
    if i=="Na":
        PROTEINA5.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA5=PROTEINA5+[nuevo]
            
print(PROTEINA5)
len(PROTEINA5)


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA6=[]

for i in urls[500:600]:
    if i=="Na":
        PROTEINA6.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA6=PROTEINA6+[nuevo]
            
print(PROTEINA6)
print(len(PROTEINA6))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA7=[]

for i in urls[600:700]:
    if i=="Na":
        PROTEINA7.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA7=PROTEINA7+[nuevo]
            
print(PROTEINA7)
print(len(PROTEINA7))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

PROTEINA8=[]

for i in urls[700:818]:
    if i=="Na":
        PROTEINA8.append("Na")
#print(PROTEINA)
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h1 property="name">([a-zA-Z0-9].*?)<', html) 
        for link in links:
            nuevo=link.decode()
            PROTEINA8=PROTEINA8+[nuevo]
            
print(PROTEINA8)
len(PROTEINA8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

PROTEINA = PROTEINA1 + PROTEINA2 + PROTEINA3 + PROTEINA4 + PROTEINA5 + PROTEINA6 + PROTEINA7 + PROTEINA8
print(PROTEINA) # SE MUESTRA TODOS LOS DATOS DE LA COLUMNA PROTEINA 
len(PROTEINA)  # LAS 818 PRIMERAS FILAS


['Na', 'Ubiquitin carboxyl-terminal hydrolase 33', 'Na', 'Na', 'Na', 'Na', 'Na', 'Integrin-associated protein', 'Intraflagellar transport protein 57 homolog', 'HERV-H LTR-associating 2', 'Myosin heavy chain 15', 'Myc proto-oncogene protein', 'RING-type E3 ubiquitin transferase', 'Phosphatidylinositol 3-kinase regulatory subunit alpha', 'CW-type domain-containing protein', 'Na', 'Na', 'Na', 'SAP domain-containing protein', 'Polycomb group RING finger protein 1', 'Na', 'Na', 'Mu-type opioid receptor', 'Collectin-11', 'Receptor transporter protein 1', 'Na', 'ST6 beta-galactoside alpha-2,6-sialyltransferase 1', 'Na', 'Adiponectin', 'Replication factor C subunit 4', 'Eukaryotic initiation factor 4A-II', 'Na', 'Kininogen-1', 'Histidine-rich glycoprotein', 'Fetuin-B', 'Alpha-2-HS-glycoprotein', 'Na', 'DnaJ homolog subfamily B member 11', 'TBCC domain-containing protein 1', 'Gamma-crystallin S', 'Diacylglycerol kinase', 'ETS domain-containing protein', 'Na', 'Transformer-2 protein homolog beta

818

In [318]:
#######################################################
# COLUMNA5 Gene                                       #
#######################################################

##################################
# PRIMERAS 100 FILAS             #
##################################


#import urllib.request, urllib.parse, urllib.error #(solicitud,analizar,error)
#import re
#import ssl  
ctx = ssl.create_default_context()                    #crear un contexto predeterminado
ctx.check_hostname = False                            #comprobar el nombre de host
ctx.verify_mode = ssl.CERT_NONE  

GENES1=[] # CREAMOS UNA LISTA VACIA DONDE ALMACENAREMOS LAS 100 PRIMERAS FILAS.

for i in urls[0:100]: # SE REALIZA LA BÚSQUEDA DE CADA ELEMENTO DE LA LISTA urls
    if i=="Na": # SI NO HAY url SE AGREGA COMO UN "Na" A LA LISTA.
        GENES1.append("Na")
    else:        # EN CASO CONTRARIO REALIZA LO SIGUIENTE:
        url = i  
        html = urllib.request.urlopen(url, context=ctx).read()  # SE ALMACENA TODA LA ESTRUCTURA DEL html EN LA VARIABLE html.
        links = re.findall(b'<h2>([A-Z].*?)<', html)  # SE REALIAZ LA BÚSQUEDA

        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode() # SE REALIZA LA DECODIFICACIÓN POR CADA link Y SE ALMACENA EN LA VARIABLE nuevo. 
                GENES1=GENES1+[nuevo] # SE ALMACENA A LA LISTA "GENES1"
        else:
        
            GENES1=GENES1+["N/A"] # EN CASO DE NO TENER "Gene" SE ALMACENA COMO "N/A"
print(GENES1) # MUESTRA LOS 100 PRIMEROS GENES
len(GENES1)
'''
# ESTOS PASOS SE REALIZAN PARA LAS SIGUIENTES FILAS.

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES2=[]

for i in urls[100:200]:
    if i=="Na":
        GENES2.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES2=GENES2+[nuevo]
        else:
        
            GENES2=GENES2+["N/A"]
print(GENES2)
print(len(GENES2))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES3=[]

for i in urls[200:300]:
    if i=="Na":
        GENES3.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES3=GENES3+[nuevo]
        else:
        
            GENES3=GENES3+["N/A"]
print(GENES3)
len(GENES3)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES4=[]

for i in urls[300:400]:
    if i=="Na":
        GENES4.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES4=GENES4+[nuevo]
        else:
        
            GENES4=GENES4+["N/A"]
print(GENES4)
print(len(GENES4))

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES5=[]

for i in urls[400:500]:
    if i=="Na":
        GENES5.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES5=GENES5+[nuevo]
        else:
        
            GENES5=GENES5+["N/A"]
print(GENES5)
len(GENES5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES6=[]

for i in urls[500:600]:
    if i=="Na":
        GENES6.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES6=GENES6+[nuevo]
        else:
        
            GENES6=GENES6+["N/A"]
print(GENES6)
len(GENES6)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES7=[]

for i in urls[600:700]:
    if i=="Na":
        GENES7.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES7=GENES7+[nuevo]
        else:
        
            GENES7=GENES7+["N/A"]
print(GENES7)
len(GENES7)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GENES8=[]

for i in urls[700:818]:
    if i=="Na":
        GENES8.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<h2>([A-Z].*?)<', html) 
        #print(links)
        if len(re.findall(b'<h2>([A-Z].*?)<', html))==1:  # Algunos Entrys no tienen el nombre de Gene, por lo tanto links saldría vacío, con len =0
            for link in links:
                nuevo=link.decode()
                GENES8=GENES8+[nuevo]
        else:
        
            GENES8=GENES8+["N/A"]
print(GENES8)
len(GENES8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

GENES = GENES1 + GENES2 + GENES3 + GENES4 + GENES5 + GENES6 + GENES7 + GENES8
print(GENES) # AL FINAL JUNTAMOS LAS 818 PRIMERAS FILAS DE LA COLUMNA GENES
len(GENES)


['Na', 'USP33', 'Na', 'Na', 'Na', 'Na', 'Na', 'CD47', 'IFT57', 'HHLA2', 'MYH15', 'MYC', 'DZIP3', 'PIK3R1', 'MORC1', 'Na', 'Na', 'Na', 'DPPA2', 'PCGF1', 'Na', 'Na', 'OPRM1', 'COLEC11', 'RTP1', 'Na', 'ST6GAL1', 'Na', 'ADIPOQ', 'RFC4', 'EIF4A2', 'Na', 'KNG1', 'HRG', 'FETUB', 'AHSG', 'Na', 'DNAJB11', 'TBCCD1', 'CRYGS', 'DGKG', 'ETV5', 'Na', 'TRA2B', 'Na', 'DHX9', 'GJA1', 'LIPC', 'TMEM41A', 'MAP3K13', 'EHHADH', 'EPHB1', 'LPO', 'CDK2', 'ANAPC13', 'AMOTL2', 'RYK', 'SLCO2A1', 'RAB6B', 'LALBA', 'LOC525947', 'SMARCA4', 'RIPK4', 'PRDM15', 'C2CD2L', 'ZBTB21', 'UMODL1', 'CAV1', 'TFF3', 'TFF2', 'ESR1', 'TMPRSS3', 'UBASH3A', 'RSPH1', 'SLC37A1', 'PDE9A', 'WDR4', 'NDUFV3', 'PKNOX1', 'IMPDH2', 'U2AF1', 'CRYAA', 'ATP1A1', 'RAD51', 'PPP1CB', 'PDXK', 'CSTB', 'C1QBP', 'AGPAT3', 'KCNH7', 'RBM4', 'IFIH1', 'FAP', 'GCG', 'DPP4', 'SLC4A10', 'TBR1', 'ADRM1', 'Na', 'ZC3H12A', 'NXPH2', 'SPOP', 'HNMT', 'THSD7B', 'CXCR4', 'DARS1', 'MCM6', 'LECT2', 'VCP', 'R3HDM1', 'Na', 'ZRANB3', 'RAB3GAP1', 'MAP3K19', 'CDK9', 'ACMSD

818

In [293]:
################################
# COLUMNA6 ORGANISMO  
################################

##################################
# PRIMERAS 100 FILAS             #
##################################

import urllib.request, urllib.parse, urllib.error #(solicitud,analizar,error)
import re
import ssl  

ctx = ssl.create_default_context()                    #crear un contexto predeterminado
ctx.check_hostname = False                            #comprobar el nombre de host
ctx.verify_mode = ssl.CERT_NONE 

ORGANISMO1=[] # CREAMOS UNA LISTA VACIA DONDE IREMOS ALMACENANDO LO QUE NECESITAMOS.

for i in urls[0:100]: # SE REALIZA LA BUSQUEDA A CADA ELEMENTO DE urls
    if i=="Na": # SI NO HAY urls SE AGREGA COMO "Na"
        ORGANISMO1.append("Na") 
    else:        
        url = i  # CASO CONTRARIO REALIZA LO SIGUIENTE:
        html = urllib.request.urlopen(url, context=ctx).read()  # SE ALMACENA TODA LA ESTRUCTURA DEL html EN LA VARIABLE html.
        links = re.findall(b'<em>([A-Z].*?)</em>', html) # REALIZA LA BÚSQUEDA 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO1=ORGANISMO1+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode() # SE REALIZA LA DECODIFICACIÓN DE CADA link
                ORGANISMO1=ORGANISMO1+[nuevo] # LUEGO SE ALMACENA EN LA LISTA "ORGANISMO1"
print(ORGANISMO1) # MUESTRA LAS PRIMERAS 100 FILAS DE LA COLUMNA ORGANISMO
len(ORGANISMO1)
'''
# LOS MISMOS PASOS SE REALIZAN EN LAS SIGUIENTES FILAS.

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO2 = []
for i in urls[100:200]:
    if i=="Na":
        ORGANISMO2.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO2=ORGANISMO2+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO2=ORGANISMO2+[nuevo]
print(ORGANISMO2)
len(ORGANISMO2)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO3 = []
for i in urls[200:300]:
    if i=="Na":
        ORGANISMO3.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO3=ORGANISMO3+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO3=ORGANISMO3+[nuevo]
print(ORGANISMO3)
len(ORGANISMO3)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO4 = []
for i in urls[300:400]:
    if i=="Na":
        ORGANISMO4.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO4=ORGANISMO4+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO4=ORGANISMO4+[nuevo]
print(ORGANISMO4)
len(ORGANISMO4)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO5 = []
for i in urls[400:500]:
    if i=="Na":
        ORGANISMO5.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO5=ORGANISMO5+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO5=ORGANISMO5+[nuevo]
print(ORGANISMO5)
len(ORGANISMO5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO6 = []
for i in urls[500:600]:
    if i=="Na":
        ORGANISMO6.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO6=ORGANISMO6+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO6=ORGANISMO6+[nuevo]
print(ORGANISMO6)
len(ORGANISMO6)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO7 = []
for i in urls[600:700]:
    if i=="Na":
        ORGANISMO7.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO7=ORGANISMO7+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO7=ORGANISMO7+[nuevo]
print(ORGANISMO7)
len(ORGANISMO7)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

ORGANISMO8 = []
for i in urls[700:818]:
    if i=="Na":
        ORGANISMO8.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<em>([A-Z].*?)</em>', html) 
        if  len(links)==2:     # si hay más de un objeto en links o si hay exactamente 2.
            ORGANISMO8=ORGANISMO8+ [links[1]]   #agrega  a la lista solo el segundo objeto de la lista
        else:
            for link in links:
                nuevo=link.decode()
                ORGANISMO8=ORGANISMO8+[nuevo]
print(ORGANISMO8)
len(ORGANISMO8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

ORGANISMO = ORGANISMO1 + ORGANISMO2 + ORGANISMO3 + ORGANISMO4 + ORGANISMO5 + ORGANISMO6 + ORGANISMO7 + ORGANISMO8
print(ORGANISMO) # SE MUESTRAN LAS 818 PRIMERAS FILAS DE LA COLUMNA ORGANISMO
len(ORGANISMO)

818

In [361]:
#######################################################
# COLUMNA7 STATUS                                     #
#######################################################

##################################
# LAS PRIMERAS 100 FILAS       #
##################################

STATUS1=[] # SE CREA UNA LISTA VACIA PARA ALMACENAR LAS 100 PRIMERAS FILAS.

for i in urls[0:100]: # SE REALIZA UNA BÚSQUEDA PARA LAS 100 PRIMERAS urls
    if i=="Na": # SI NO HAY urls SE AGREGA COMO "Na"
        STATUS1.append("Na")
    else:        
        url = i 
        html = urllib.request.urlopen(url, context=ctx).read()  # SE ALMACENA TODA LA ESTRUCTURA DEL html EN LA VARIABLE html.
        links = re.findall(b'<span>([UR].*viewed)', html) # REALIZA LA BÚSQUEDA
        for link in links:
            nuevo=link.decode() # REALIZA UNA DECODIFICACIÓN PARA CADA link
            STATUS1=STATUS1+[nuevo] # SE VA AÑADIENDO A LA LISTA "STATUS1"
print(STATUS1) # MUESTRA LAS 100 PRIMERAS FILAS DE LA COLUMNA "STATUS"

len(STATUS1)

# LOS MISMOS PASOS SE REALIZAN PARA LAS SIGUIENTES FILAS.
'''
##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS2=[]

for i in urls[100:200]:
    if i=="Na":
        STATUS2.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS2=STATUS2+[nuevo]
print(STATUS2)

len(STATUS2)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS3=[]

for i in urls[200:300]:
    if i=="Na":
        STATUS3.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS3=STATUS3+[nuevo]
print(STATUS3)

len(STATUS3)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS4=[]

for i in urls[300:400]:
    if i=="Na":
        STATUS4.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS4=STATUS4+[nuevo]
print(STATUS4)

len(STATUS4)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS5=[]

for i in urls[400:500]:
    if i=="Na":
        STATUS5.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS5=STATUS5+[nuevo]
print(STATUS5)

len(STATUS5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS6=[]

for i in urls[500:600]:
    if i=="Na":
        STATUS6.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS6=STATUS6+[nuevo]
print(STATUS6)

len(STATUS6)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS7=[]

for i in urls[600:700]:
    if i=="Na":
        STATUS7.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS7=STATUS7+[nuevo]
print(STATUS7)

len(STATUS7)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

STATUS8=[]

for i in urls[700:818]:
    if i=="Na":
        STATUS8.append("Na")
    else:        
        url = i
        html = urllib.request.urlopen(url, context=ctx).read()  
        links = re.findall(b'<span>([UR].*viewed)', html) 
        for link in links:
            nuevo=link.decode()
            STATUS8=STATUS8+[nuevo]
print(STATUS8)

len(STATUS8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

STATUS = STATUS1 + STATUS2 + STATUS3 + STATUS4 + STATUS5 + STATUS6 + STATUS7 + STATUS8
print(STATUS) # MUESTRA LAS 818 PRIMERAS FILAS DE LA COLUMNA STATUS
len(STATUS)

818

In [420]:
#######################################################
# COLUMNA8 GO - Biological process                    #
#######################################################

##################################
# LAS PRIMERAS 100 FILAS         #
##################################

GO_Biological_process1 = [] # CREAMOS UNA LISTA VACIA PARA LAS 100 PRIMERAS FILAS

for i in urls[0:100]: # SE REALIZA LA BUSQUEDA DE LAS 100 PRIMERAS urls
    if i=="Na":  # SI NO HAY url SE AGREGA A LA LISTA COMO Na
        GO_Biological_process1.append("Na") 
    else: # EN CASO CONTRARIO REALIZA LO SIGUIENTE:
        uclient=urlopen(i)  # SE ABRE EL URL 
        pagehtml=uclient.read() # SE REALIZA LA LECTURA A LA PÁGINA
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")  # ALMACENA LA ESTRUCTURA LA url EN pagesoup
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) # De la estructura almacenada en pagesoup con el find
                                                             # se busca la etiqueta "ul" que tenga la clase:noNumbering biological_process y se almacena en page.
        if page==None: # SI page NO TIENE INFORMACION SE MUESTRA COMO "None" Y EN LA LISTA SE ALMACENA COMO "Na"
            GO_Biological_process1.append("Na")
        else: # EN CASO DE TENER INFORMACION REALIZA LO SIGUIENTE:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            # EN location SE ALMACENA COMO LISTA EN UNA VARIABLE location LA INFORMACION QUE SE ENCUENTRA EN LA ETIQUETA "a" CON 
            # onclick:window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"
            
            cadena = "" # CREAMOS UNA CADENA PARA IR ALMACENANDO LA INFORMACION QUE SE ENCUENTRE.
            for i in range(0,len(location)): # se realiza un for para almacenar cada caracteristica de GO - Biological process,
                if location[i].text=="": # si no hay texto dentro de "location" se agrega "-"
                    cadena = cadena + "-"
                else: # en caso contrario realiza lo siguiente:

                    cadena = cadena + location[i].text + "." # si hay texto va agregando a la cadena mas un "." que seria como una separacion con el siguiente

            GO_Biological_process1.append(cadena) # cuando termine el ciclo for recien se agrega la cadena a la lista GO_Biological_process1

print(GO_Biological_process1) # MUESTRA LAS 100 PRIMERAS FILAS DE LA COLUMNA GO_Biological_process
len(GO_Biological_process1)
'''
# LOS MISMOS PASOS SE REALIZAN PARA LAS SIGUIENTES FILAS

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process2 = []

for i in urls[100:200]:
    if i=="Na":
        GO_Biological_process2.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process2.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process2.append(cadena)

print(GO_Biological_process2)
len(GO_Biological_process2)


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process3 = []

for i in urls[200:300]:
    if i=="Na":
        GO_Biological_process3.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process3.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process3.append(cadena)

print(GO_Biological_process3)
len(GO_Biological_process3)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process4 = []

for i in urls[300:400]:
    if i=="Na":
        GO_Biological_process4.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process4.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process4.append(cadena)

print(GO_Biological_process4)
len(GO_Biological_process4)


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process5 = []

for i in urls[400:500]:
    if i=="Na":
        GO_Biological_process5.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process5.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process5.append(cadena)

print(GO_Biological_process5)
len(GO_Biological_process5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process6 = []

for i in urls[500:600]:
    if i=="Na":
        GO_Biological_process6.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process6.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process6.append(cadena)

print(GO_Biological_process6)
len(GO_Biological_process6)


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process7 = []

for i in urls[600:700]:
    if i=="Na":
        GO_Biological_process7.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process7.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process7.append(cadena)

print(GO_Biological_process7)
len(GO_Biological_process7)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

GO_Biological_process8 = []

for i in urls[700:818]:
    if i=="Na":
        GO_Biological_process8.append("Na")
    else:
        uclient=urlopen(i)
        pagehtml=uclient.read()
        uclient.close()                                    
        pagesoup=soup(pagehtml, "html.parser")
        page=pagesoup.find("ul",{"class":"noNumbering biological_process"}) 
        
        if page==None:
            GO_Biological_process8.append("Na")
        else:
            location=page.find_all("a",{"onclick":"window.ga('UniProt-Entry-View', 'click', 'Display-GO-Term');"})
            #print(location)
            cadena = ""
            for i in range(0,len(location)):
                if location[i].text=="":
                    cadena = cadena + "-"
                else:
                    #GO_Biological_process.append(location[i].text)
                    cadena = cadena + location[i].text + "."
                #print(type(location[i].text))

            GO_Biological_process8.append(cadena)

print(GO_Biological_process8)
len(GO_Biological_process8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

GO_Biological_process = GO_Biological_process1 + GO_Biological_process2 + GO_Biological_process3 + GO_Biological_process4 + GO_Biological_process5 + GO_Biological_process6 + GO_Biological_process7 + GO_Biological_process8
print(GO_Biological_process) # VA A IMPRIMIR LAS 818 PRIMERAS FILAS DE LA COLUMNA GO_Biological_process
len(GO_Biological_process)

818

In [496]:
##############################################################
# CON LAS PRIMERAS 8 COLUMNAS CREAMOS UN DATA FRAME          #
##############################################################

import pandas as pd
##Guardando un archivo de formato data frame
#Creamos un diccionario data
data = {'Gen abrev': gen_abreviado1,
        'Entry': LISTA_ENTRY,
        'Entry name': LISTA_ENTRY_NAME,
        'Protein' : PROTEINA,
        'Gene': GENES,
        'Organism' : ORGANISMO,
        'Status': STATUS,
        'GO-Biological process': GO_Biological_process}
# Luego creamos un dataframe y especificamos las columnas.



In [497]:
df = pd.DataFrame(data, columns = ['Gen abrev','Entry','Entry name','Protein','Gene','Organism','Status', 'GO-Biological process'])

df

,Gen abrev,Entry,Entry name,Protein,Gene,Organism,Status,GO-Biological process
0,TRNAG-UCC,Na,Na,Na,Na,Na,Na,Na
1,ROBO1,A6QNM7,UBP33_BOVIN,Ubiquitin carboxyl-terminal hydrolase 33,USP33,Bos taurus (Bovine),Reviewed,axon guidance.cell migration.cellular response...
2,TRNAC-GCA,Na,Na,Na,Na,Na,Na,Na
3,TRNAC-ACA,Na,Na,Na,Na,Na,Na,Na
4,MIR2286,Na,Na,Na,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...
813,TUBA1B,A0A452DIK7,A0A452DIK7_BOVIN,Tubulin alpha chain,TUBA1B,Bos taurus (Bovine),Unreviewed,cellular response to interleukin-4.microtubule...
814,LMBR1L,P02754,LACB_BOVIN,Beta-lactoglobulin,LGB,Bos taurus (Bovine),Reviewed,Na
815,DHH,F1MFP2,F1MFP2_BOVIN,Hedgehog protein,DHH,Bos taurus (Bovine),Unreviewed,cell-cell signaling.cell fate specification.in...
816,RHEBL1,Q7YS69,REBL1_BOVIN,GTPase RhebL1,RHEBL1,Bos taurus (Bovine),Reviewed,small GTPase mediated signal transduction.


In [498]:
#####################################################
# CREAMOS UN EXCEL CON LAS 8 COLUMNAS CREADAS       #
#####################################################

#### creación del excel
from pandas import ExcelWriter # Importamos la libreria ExcelWriter
writer = ExcelWriter('./PARTE_1.xlsx') # el nombre del excel seria "PARTE_1"
df.to_excel(writer, index=False)
writer.save()


In [148]:
#######################################################
# COLUMNA9 MOLECULAR FUNCTION                         #
#######################################################

##################################
# LAS PRIMERAS 100 FILAS         #
##################################

MOLECULAR_FUNCTION1=[] # CREAMOS UNA LISTA VACIA PARA ALMACENAR LAS 100 PRIMERAS FILAS DE MOLECULAR_FUNCTION

for link in urls[0:100]: # REALIZA UN RECORRIDO POR LAS urls
    if link=="Na": # si no encuentra la "url" AGREGA A LA LISTA COMO "Na"
        MOLECULAR_FUNCTION1.append("Na")
    else: # EN CASO DE ENCONTRAR "url" REALIZA LO SIGUIENTE:
        
        uclient=urlopen(link) # ABRE LA "url"
        pagehtml=uclient.read() # LEE LA PAGINA
        uclient.close() 
        pagesoup=soup(pagehtml, "html.parser") # ALMACENA LA ESTRUCTURA LA url EN pagesoup
        
        if pagesoup.find("div",{"id":"function"}): # DE LA ESTRUCTURA BUSCA LA ETIQUETA "div" con id=function
            section=pagesoup.find("div",{"id":"function"})  # si encuentra la estructura lo almacena en la variable section.
            if section.find("table",{"class":"databaseTable"}): # si en la variable section se encuentra la etiqueta table con class=databaseTable realiza lo siguiente:
                tabla=section.find("table",{"class":"databaseTable"}) # de la variable section busca la etiqueta "table" con "class=databaseTable" y lo almacena en la variable tabla
                span=tabla.find("tbody") # de la variable tabla busca la etiqueta "tbody" y el contenido los almacena en la variable "span"
                td=tabla.find_all("td")  # de la variable "span" va  a buscar la etiqueta "td"y la va a almacenar como lista en la variable "td"
                for i in range(len(td)): # luego realizamos un for desde: 0 hasta la longitud de "td"
                    if td[i].text == "Molecular function": # LUEGO DE CADA ELEMENTO DE "td" EXTRAEMOS EL TEXTO Y SI ES IGUAL A "Molecular function" VA A REALIZAR LO SIGUIENTE:
                        span=td[i+1].find_all("span",recursive=False) # HACEMOS td[i+1] PARA QUE EN EL SIGUIENTE "td" BUSQUE LA ETIQUETA "span" Y LO ALMACENE COMO LISTA
                                                                      # EN LA NUEVA VARIABLE "span", CON "recursive=False" YA NO PROFUNDICE EL CONTENIDO DE LA ETIQUETA "span"
                        span_0=span[0].find_all("a",recursive=False) # LUEGO CREAMOS UNA NUEVA VARIABLE DONDE SE VA A ALMACENAR LA BUSQUEDA QUE SE REALIZA CON EL PRIMER "span"
                                                                     # DONDE SE VA A BUSCAR LA ETIQUETA "a", "recursive=False" YA NO PROFUNDICE EL CONTENIDO DE LA ETIQUETA "a"
                        string="" # CREAMOS UNA CADENA NUEVA
                        for L in span_0: # POR CADA ELEMENTO DE span_0 SE VA A REALIZAR LO SIGUIENTE:
                            string= string  + L.text + "." # SE EXTRAE EL TEXTO DE CADA ELEMENTO DE "span_0" Y SE VA ALMACENANDO EN "string"

                        MOLECULAR_FUNCTION1.append(string) # AL TERMINAR EL FOR RECIEN SE AÑADE LOS ELEMENTOS DE LA CADENA A LA LISTA MOLECULAR_FUNCTION1
                        break  #  SI YA ENCONTRO QUE td[i]="Molecular function" EL BUCLE TERMINA YA QUE YA NO NECESITAMOS EL RESTO.                
                    else:
                        if  i != (len(td)-1): # SI EL "i" ES DIFERENTE DE LA LONGITUD DE "td"-1
                            continue # EL BUCLE CONTINUA HASTA QUE SEAN IGUALES Y PASA AL SIGUIENTE "else"
                        else:
                            MOLECULAR_FUNCTION1.append('Na') # CUANO CUMPLE LO ANTERIOR, A LA LISTA MOLECULAR_FUNCTION1 SE LE AGREGA EL "Na"
            else :
                MOLECULAR_FUNCTION1.append("Na") # SI EN LA VARIABLE "section" NO SE ENCUENTRA LA ETIQUETA "table" con "class=databaseTable"
                                                # SE AÑADE A LA LISTA MOLECULAR_FUNCTION1 "Na".
        else:
            MOLECULAR_FUNCTION1.append("Na") # SI DE LA VARIABLE "pagesoup" NO ENCUENTRA LA ETIQUETA "div" con "id=function"
                                            # SE AÑADE A LA LISTA MOLECULAR_FUNCTION1 "Na"
        
print(MOLECULAR_FUNCTION1) # VA A MOSTRAR LAS 100 PRIMERAS FILAS CON LOS DATOS SOLICITADOS DE LA COLUMNA MOLECULAR_FUNCTION.
len(MOLECULAR_FUNCTION1)

# EL MISMO PROCESO SE REALIZA PARA LAS SIGUIENTES FILAS.
'''
##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION2=[]

for link in urls[100:200]:
    if link=="Na":
        MOLECULAR_FUNCTION2.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION2.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION2.append('Na') 
            else :
                MOLECULAR_FUNCTION2.append("Na")
        else:
            MOLECULAR_FUNCTION2.append("Na")
        
print(MOLECULAR_FUNCTION2)
len(MOLECULAR_FUNCTION2)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION3=[]

for link in urls[200:300]:
    if link=="Na":
        MOLECULAR_FUNCTION3.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION3.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION3.append('Na') 
            else :
                MOLECULAR_FUNCTION3.append("Na")
        else:
            MOLECULAR_FUNCTION3.append("Na")
        
print(MOLECULAR_FUNCTION3)
len(MOLECULAR_FUNCTION3)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION4=[]

for link in urls[300:400]:
    if link=="Na":
        MOLECULAR_FUNCTION4.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION4.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION4.append('Na') 
            else :
                MOLECULAR_FUNCTION4.append("Na")
        else:
            MOLECULAR_FUNCTION4.append("Na")
        
print(MOLECULAR_FUNCTION4)
len(MOLECULAR_FUNCTION4)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION5=[]

for link in urls[400:500]:
    if link=="Na":
        MOLECULAR_FUNCTION5.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION5.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION5.append('Na') 
            else :
                MOLECULAR_FUNCTION5.append("Na")
        else:
            MOLECULAR_FUNCTION5.append("Na")
        
print(MOLECULAR_FUNCTION5)
len(MOLECULAR_FUNCTION5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION5=[]

for link in urls[400:500]:
    if link=="Na":
        MOLECULAR_FUNCTION5.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION5.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION5.append('Na') 
            else :
                MOLECULAR_FUNCTION5.append("Na")
        else:
            MOLECULAR_FUNCTION5.append("Na")
        
print(MOLECULAR_FUNCTION5)
len(MOLECULAR_FUNCTION5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION6=[]

for link in urls[500:600]:
    if link=="Na":
        MOLECULAR_FUNCTION6.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION6.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION6.append('Na') 
            else :
                MOLECULAR_FUNCTION6.append("Na")
        else:
            MOLECULAR_FUNCTION6.append("Na")
        
print(MOLECULAR_FUNCTION6)
len(MOLECULAR_FUNCTION6)


##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION7=[]

for link in urls[600:700]:
    if link=="Na":
        MOLECULAR_FUNCTION7.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION7.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION7.append('Na') 
            else :
                MOLECULAR_FUNCTION7.append("Na")
        else:
            MOLECULAR_FUNCTION7.append("Na")
        
print(MOLECULAR_FUNCTION7)
len(MOLECULAR_FUNCTION7)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

MOLECULAR_FUNCTION8=[]

for link in urls[700:818]:
    if link=="Na":
        MOLECULAR_FUNCTION8.append("Na")
    else:
        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            if section.find("table",{"class":"databaseTable"}): 
                tabla=section.find("table",{"class":"databaseTable"})
                span=tabla.find("tbody")
                td=tabla.find_all("td")
                for i in range(len(td)):
                    if td[i].text == "Molecular function":
                        span=td[i+1].find_all("span",recursive=False)
                        span_0=span[0].find_all("a",recursive=False)
                        string=""
                        for L in span_0:
                            string= string  + L.text + "."

                        MOLECULAR_FUNCTION8.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            #print(i)
                            continue
                        else:
                            MOLECULAR_FUNCTION8.append('Na') 
            else :
                MOLECULAR_FUNCTION8.append("Na")
        else:
            MOLECULAR_FUNCTION8.append("Na")
        
print(MOLECULAR_FUNCTION8)
len(MOLECULAR_FUNCTION8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

MOLECULAR_FUNCTION = MOLECULAR_FUNCTION1 + MOLECULAR_FUNCTION2 + MOLECULAR_FUNCTION3 + MOLECULAR_FUNCTION4 + MOLECULAR_FUNCTION5 + MOLECULAR_FUNCTION6 + MOLECULAR_FUNCTION7 + MOLECULAR_FUNCTION8
print(MOLECULAR_FUNCTION) # VA A MOSTRAR LAS PRIMERAS 818 FILAS DE LA COLUMNA MOLECULAR FUNCTION
len(MOLECULAR_FUNCTION)

818

In [234]:
#######################################################
# COLUMNA9 BIOLOGICAL PROCESS                         #
#######################################################

##################################
# LAS PRIMERAS 100 FILAS         #
##################################

BIOLOGICAL_PROCESS1=[] # CREAMOS UNA LISTA VACIA PARA ALMACENAR LAS 100 PRIMERAS FILAS DE BIOLOGICAL PROCESS
 
for link in urls[0:100]: # REALIZA UN RECORRIDO POR LAS urls
    
    if link=="Na": # si no encuentra la "url" AGREGA A LA LISTA COMO "Na"
        BIOLOGICAL_PROCESS1.append("Na")        
    else:   # EN CASO DE ENCONTRAR "url" REALIZA LO SIGUIENTE:     
        uclient=urlopen(link)  # ABRE LA "url"
        pagehtml=uclient.read()  # LEE LA PAGINA
        uclient.close() 
        pagesoup=soup(pagehtml, "html.parser") # ALMACENA LA ESTRUCTURA LA url EN pagesoup
        
        if pagesoup.find("div",{"id":"function"}): # DE LA ESTRUCTURA BUSCA LA ETIQUETA "div" con id=function
            section=pagesoup.find("div",{"id":"function"}) # si encuentra la estructura lo almacena en la variable section.
             
            if section.find("table",{"class":"databaseTable"}): # si en la variable section se encuentra la etiqueta table con class=databaseTable realiza lo siguiente:
               
                tabla=section.find("table",{"class":"databaseTable"})  # de la variable section busca la etiqueta "table" con "class=databaseTable" y lo almacena en la variable tabla
                td=tabla.find_all("td")  # de la variable "tabla" va  a buscar la etiqueta "td"y la va a almacenar como lista en la variable "td"
                
                for i in range(len(td)): # luego realizamos un for desde: 0 hasta la longitud de "td"
                    if td[i].text == "Biological process": # LUEGO DE CADA ELEMENTO DE "td" EXTRAEMOS EL TEXTO Y SI ES IGUAL A "Biological process" VA A REALIZAR LO SIGUIENTE:
                        index= i+1  # colocamos un contador para que realize las siguientes busquedas                                      
                        span=td[index].find_all("span",recursive=False) # por cada td[index] va a buscar la etiqueta "span" y lo va almacenar como lista en la variable "span"                         
                        span_0=span[0].find_all("a",recursive=False) # luego de la variable "span" anterior va a realizar una busqueda de las etiquetas "a"
                                                                     # y las va a almacenar como lista en "span_0", "recursive=False": es para que no profundice la etiqueta "a"
                        string="" # CREAMOS UNA CADENA VACIA "string"

                        for L in span_0:  # PARA CADA ELEMENTO DE LA VARIABLE "span_0" SE REALIZA LO SIGUIENTE:                       
                            string= string  + L.text +  "." # SE VA A EXTRAER EL TEXTO DEL ELEMENTO DE "span_0" y se va a ir agregando
                                                            # a la variable "string" con "." como separador.

                        BIOLOGICAL_PROCESS1.append(string) # AL TERMINAR DE REALIZAR EL FOR RECIEN SE VA A AGREGAR A LA LISTA BIOLOGICAL_PROCESS1
                        break     #  como td[i].text == "Biological process" EL BUCLE TERMINA PORQUE EL RESTO YA NO ES IMPORTANTE             
                    else: 
                        if  i != (len(td)-1): # SI "i" es diferente de (len(td)-1) va a continuar hasta que encuentre la igualad
                            continue          # y recien pasara al "else".
                        else: 
                            BIOLOGICAL_PROCESS1.append('Na') # el caso de que cumpla la igualdad se va a agregar a la lista BIOLOGICAL_PROCESS1 un "Na"                                                     
            else :
                BIOLOGICAL_PROCESS1.append("Na") # en caso de no cumplir el if se adiciona "Na"  a la lista BIOLOGICAL_PROCESS
        else:            
            BIOLOGICAL_PROCESS1.append("Na") # en caso de no cumplir el if se adiciona "Na"  a la lista BIOLOGICAL_PROCESS
        
print(BIOLOGICAL_PROCESS1) # muestra las 100 primeras filas de la columna .
len(BIOLOGICAL_PROCESS1)
'''
# EL MISMO PASO SE REALIZA PARA LAS SIGUIENTES FILAS

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS2=[]

for link in urls[100:200]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS2.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS2.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS2.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS2.append("Na")
        else:            
            BIOLOGICAL_PROCESS2.append("Na")
        
print(BIOLOGICAL_PROCESS2)
len(BIOLOGICAL_PROCESS2)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS3=[]

for link in urls[200:300]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS3.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS3.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS3.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS3.append("Na")
        else:            
            BIOLOGICAL_PROCESS3.append("Na")
        
print(BIOLOGICAL_PROCESS3)
len(BIOLOGICAL_PROCESS3)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS4=[]

for link in urls[300:400]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS4.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS4.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS4.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS4.append("Na")
        else:            
            BIOLOGICAL_PROCESS4.append("Na")
        
print(BIOLOGICAL_PROCESS4)
len(BIOLOGICAL_PROCESS4)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS5=[]

for link in urls[400:500]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS5.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS5.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS5.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS5.append("Na")
        else:            
            BIOLOGICAL_PROCESS5.append("Na")
        
print(BIOLOGICAL_PROCESS5)
len(BIOLOGICAL_PROCESS5)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS6=[]

for link in urls[500:600]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS6.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS6.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS6.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS6.append("Na")
        else:            
            BIOLOGICAL_PROCESS6.append("Na")
        
print(BIOLOGICAL_PROCESS6)
len(BIOLOGICAL_PROCESS6)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS7=[]

for link in urls[600:700]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS7.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS7.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS7.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS7.append("Na")
        else:            
            BIOLOGICAL_PROCESS7.append("Na")
        
print(BIOLOGICAL_PROCESS7)
len(BIOLOGICAL_PROCESS7)

##################################
# LAS SIGUIENTES 100 FILAS       #
##################################

BIOLOGICAL_PROCESS8=[]

for link in urls[700:818]:
    
    if link=="Na":
        BIOLOGICAL_PROCESS8.append("Na")        
    else:        
        uclient=urlopen(link)
        pagehtml=uclient.read()
        uclient.close()
        pagesoup=soup(pagehtml, "html.parser")
        
        if pagesoup.find("div",{"id":"function"}):
            section=pagesoup.find("div",{"id":"function"})
            
            if section.find("table",{"class":"databaseTable"}):
               
                tabla=section.find("table",{"class":"databaseTable"})
                td=tabla.find_all("td")
                
                for i in range(len(td)):
                    if td[i].text == "Biological process":
                        index= i+1                                        
                        span=td[index].find_all("span",recursive=False)                          
                        span_0=span[0].find_all("a",recursive=False)
                        string=""

                        for L in span_0:                          
                            string= string  + L.text +  "."

                        BIOLOGICAL_PROCESS8.append(string)
                        break                   
                    else:
                        if  i != (len(td)-1):
                            continue
                        else:
                            BIOLOGICAL_PROCESS8.append('Na')                                                       
            else :
                BIOLOGICAL_PROCESS8.append("Na")
        else:            
            BIOLOGICAL_PROCESS8.append("Na")
        
print(BIOLOGICAL_PROCESS8)
len(BIOLOGICAL_PROCESS8)
'''
#######################################################
# JUNTANDO TODAS LAS LISTAS                           #
#######################################################

BIOLOGICAL_PROCESS = BIOLOGICAL_PROCESS1 + BIOLOGICAL_PROCESS2 + BIOLOGICAL_PROCESS3 + BIOLOGICAL_PROCESS4 + BIOLOGICAL_PROCESS5 + BIOLOGICAL_PROCESS6 + BIOLOGICAL_PROCESS7 + BIOLOGICAL_PROCESS8
#print(BIOLOGICAL_PROCESS)
len(BIOLOGICAL_PROCESS)

818

In [235]:
datos = pd.read_excel("PARTE_1.xlsx")

In [236]:
datos

,Gen abrev,Entry,Entry name,Protein,Gene,Organism,Status,GO-Biological process
0,TRNAG-UCC,Na,Na,Na,Na,Na,Na,Na
1,ROBO1,A6QNM7,UBP33_BOVIN,Ubiquitin carboxyl-terminal hydrolase 33,USP33,Bos taurus (Bovine),Reviewed,axon guidance.cell migration.cellular response...
2,TRNAC-GCA,Na,Na,Na,Na,Na,Na,Na
3,TRNAC-ACA,Na,Na,Na,Na,Na,Na,Na
4,MIR2286,Na,Na,Na,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...
813,TUBA1B,A0A452DIK7,A0A452DIK7_BOVIN,Tubulin alpha chain,TUBA1B,Bos taurus (Bovine),Unreviewed,cellular response to interleukin-4.microtubule...
814,LMBR1L,P02754,LACB_BOVIN,Beta-lactoglobulin,LGB,Bos taurus (Bovine),Reviewed,Na
815,DHH,F1MFP2,F1MFP2_BOVIN,Hedgehog protein,DHH,Bos taurus (Bovine),Unreviewed,cell-cell signaling.cell fate specification.in...
816,RHEBL1,Q7YS69,REBL1_BOVIN,GTPase RhebL1,RHEBL1,Bos taurus (Bovine),Reviewed,small GTPase mediated signal transduction.


In [238]:
data = pd.DataFrame(datos)
print(data)

     Gen abrev       Entry        Entry name  \
0    TRNAG-UCC          Na                Na   
1        ROBO1      A6QNM7       UBP33_BOVIN   
2    TRNAC-GCA          Na                Na   
3    TRNAC-ACA          Na                Na   
4      MIR2286          Na                Na   
..         ...         ...               ...   
813     TUBA1B  A0A452DIK7  A0A452DIK7_BOVIN   
814     LMBR1L      P02754        LACB_BOVIN   
815        DHH      F1MFP2      F1MFP2_BOVIN   
816     RHEBL1      Q7YS69       REBL1_BOVIN   
817      KMT2D      P49884        ESR1_BOVIN   

                                      Protein    Gene             Organism  \
0                                          Na      Na                   Na   
1    Ubiquitin carboxyl-terminal hydrolase 33   USP33  Bos taurus (Bovine)   
2                                          Na      Na                   Na   
3                                          Na      Na                   Na   
4                                

### AGREGANDO LAS DOS COLUMNAS RESTANTES 

In [243]:
data["Molecular function"]=MOLECULAR_FUNCTION

In [245]:
data["Biological process"]=BIOLOGICAL_PROCESS

In [246]:
data

,Gen abrev,Entry,Entry name,Protein,Gene,Organism,Status,GO-Biological process,Molecular function,Biological process
0,TRNAG-UCC,Na,Na,Na,Na,Na,Na,Na,Na,Na
1,ROBO1,A6QNM7,UBP33_BOVIN,Ubiquitin carboxyl-terminal hydrolase 33,USP33,Bos taurus (Bovine),Reviewed,axon guidance.cell migration.cellular response...,Hydrolase.Protease.Thiol protease.,Endocytosis.Ubl conjugation pathway.
2,TRNAC-GCA,Na,Na,Na,Na,Na,Na,Na,Na,Na
3,TRNAC-ACA,Na,Na,Na,Na,Na,Na,Na,Na,Na
4,MIR2286,Na,Na,Na,Na,Na,Na,Na,Na,Na
...,...,...,...,...,...,...,...,...,...,...
813,TUBA1B,A0A452DIK7,A0A452DIK7_BOVIN,Tubulin alpha chain,TUBA1B,Bos taurus (Bovine),Unreviewed,cellular response to interleukin-4.microtubule...,Na,Na
814,LMBR1L,P02754,LACB_BOVIN,Beta-lactoglobulin,LGB,Bos taurus (Bovine),Reviewed,Na,Milk protein.,Transport.
815,DHH,F1MFP2,F1MFP2_BOVIN,Hedgehog protein,DHH,Bos taurus (Bovine),Unreviewed,cell-cell signaling.cell fate specification.in...,Developmental protein.Hydrolase.Protease.,Na
816,RHEBL1,Q7YS69,REBL1_BOVIN,GTPase RhebL1,RHEBL1,Bos taurus (Bovine),Reviewed,small GTPase mediated signal transduction.,Na,Na


### CREACIÓN DEL EXCEL CON LAS 10 COLUMNAS

In [248]:
#### creación del excel
from pandas import ExcelWriter
writer = ExcelWriter('./PARTE_1_completa.xlsx')
data.to_excel(writer, index=False)
writer.save()